# 08 - Tower Statics

## Rigid-body *statics without* block friction

Static equilibrium (absence external disturbances) is the most fundamental property that the tower must have in order for it to not topple. To start, masses and inertias of the blocks will be used to compute the net wrench
(force and moment) due to gravity, without considering frictional effects or minor deviations in the blocks’ nominal dimensions.

In [67]:
import numpy as np

# block mass
m = 0.01    # kg
# block length, width, and height
l = 0.1     # m
w = 0.01     # m
h = 0.01     # m
K0 = int(18) # initial tower height
J = int(3) # tower width
n = J*K0 # total number of blocks

# generate block "serial numbers"
block_id = np.random.randint(low=10000,high=99999,size=(n,))

def initialize_tower(l,w,h,K0,J):
    # initial tower state X
    tower_state = [[{} for _ in range(J)] for _ in range(K0)]

    # Use a for loop to get the initial state of the tower
    count=0
    for k in range(K0):
        for j in range(J):
            if k%2 == 1:
                x_cm = 0.5*l + j*l
                y_cm = 0.5*w + j*w
                #
                x1 = j*l
                x2 = (j+1)*l
                #
                y1 = j*w
                y2 = (j+1)*w
                #
                #z1 = k*h
                #z2 = (k+1)*h
            else:
                x_cm = 0.5*w + j*w
                y_cm = 0.5*l + j*l
                #
                x1 = j*w
                x2 = (j+1)*w
                #
                y1 = j*l
                y2 = (j+1)*l
            edges = [(x1,y1),(x2,y1),(x1,y2),(x2,y2)]  
            z_cm = 0.5*h + k*h
            tower_state[k][j] = {'block': block_id[count],
                                 'XCM': x_cm,'YCM': y_cm, 'ZCM': z_cm,
                                 'edges': edges,
                                 'feasible': False} # may be incorrect on the first iteration
            count +=1
    return tower_state


tower_state = initialize_tower(l,w,h,K0,J)

In [68]:

def get_cm(k_low, k_high, tower_state=tower_state):
    """
    This function returns the center of mass of the tower by counting all blocks above row k in the tower
    """
    # initialize center of mass
    x_cm = 0
    y_cm = 0
    z_cm = 0
    # loop over all blocks in the tower
    count=0
    for k in range(k_low,k_high):
        for j in range(J):
            # add the contribution of the jth block to the center of mass
            if tower_state[k][j]['block'] == 0: # if there is no block,
                continue # skip this loop iteration
            x_cm += tower_state[k][j]['XCM']
            y_cm += tower_state[k][j]['YCM']
            z_cm += tower_state[k][j]['ZCM']
            count += 1
    # divide by the number of blocks to get the average
    x_cm /= count
    y_cm /= count
    z_cm /= count
    return (x_cm, y_cm, z_cm)

In [69]:
import numpy as np
from scipy.spatial import ConvexHull

def is_collinear(points):
    """
    Check if all points are collinear.

    :param points: A list of tuples [(x1, y1), (x2, y2), ...] representing the points.
    :return: Boolean indicating whether the points are collinear.
    """
    if len(points) < 3:
        return True

    x0, y0 = points[0]
    for i in range(1, len(points) - 1):
        x1, y1 = points[i]
        x2, y2 = points[i + 1]
        area = x0 * (y1 - y2) + x1 * (y2 - y0) + x2 * (y0 - y1)
        if area != 0:
            return False
    return True

def is_point_in_line_segment(point, line_points):
    """
    Check if the point is in the line segment defined by line_points.

    :param point: A tuple (x_star, y_star) representing the point to be checked.
    :param line_points: A list of two tuples [(x1, y1), (x2, y2)] representing the line segment.
    :return: Boolean indicating whether the point is in the line segment.
    """
    (x1, y1), (x2, y2) = line_points
    (x, y) = point
    if x1 != x2:
        lambda_ = (x - x1) / (x2 - x1)
    else:
        lambda_ = (y - y1) / (y2 - y1)
    return 0 <= lambda_ <= 1 and min(y1, y2) <= y <= max(y1, y2) and min(x1, x2) <= x <= max(x1, x2)

def check_point_in_hull(points, point_to_check):
    """
    Check if a point is inside the convex hull of a collection of points.

    :param points: A list of tuples [(x1, y1), (x2, y2), ...] representing the points.
    :param point_to_check: A tuple (x_star, y_star) representing the new point.
    :return: Boolean indicating whether the point is inside the convex hull.
    """
    if is_collinear(points):
        return is_point_in_line_segment(point_to_check, [min(points), max(points)])
    else:
        hull = ConvexHull(points)
        new_points = np.append(points, [point_to_check], axis=0)
        new_hull = ConvexHull(new_points)
        return list(new_hull.vertices) == list(hull.vertices)

def test_convex_hull():
    # Example usage
    points = [(1, 2), (3, 4), (5, 6), (7, 8)]
    point_to_check = (4, 5)
    result = check_point_in_hull(points, point_to_check)
    print("Point is inside the convex hull." if result else "Point is outside the convex hull.")
    return None




In [71]:
def feasible_blocks(tower_state,K=K0,J=J):
    B = [[{} for _ in range(J)] for _ in range(K)]
    for k in range(K0):
        for j in range(J):
            if tower_state[k][j]['block'] == 0:
                continue # skip this loop iteration
            point_to_check = (tower_state[k][j]['XCM'], tower_state[k][j]['YCM'])
            points = tower_state[k][j]['edges']
            B[k][j] = check_point_in_hull(points, point_to_check)  
    return B

feasible_blocks(tower_state)

[[True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True],
 [True, True, True]]